In [1]:
#package imports
import json
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\epguo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [15]:
from nltk.stem import *
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [93]:
#security related keywords
security_words = ['buffer', 'corrupt', 'boundary', 'valid', 'invalid','bypass','neutral','restrict',
                  'pathname','verifi','verif','query','request','author', 'authent','command','permission',
                  'password','key','deseri','encrypt','decrypt','credenti','passkey','depend']

In [7]:
memory_fxns = ['malloc','calloc','memcpy','free','realloc','alloc','memchr','memcmp','memmove','memset']

In [11]:
# ignore this block 
json_file = 'CVE-1999-0199.json'
with open(json_file, 'r') as file:
    data = json.load(file)

In [95]:
# test block
file_changes = data['file_changes']
tokens_added = 0
tokens_removed = 0
mem_words_present = []
sec_words_present = []
for i in file_changes:
    for j in file_changes[i]:
        if j != 'used_context_lines':
            k = 0
            while k < len(file_changes[i][j]):
                line = file_changes[i][j][k]
                #print(line)
                mem_words_present = mem_words_present + [word for word in memory_fxns if word in line]
                if( line != "" and line[0] == '-'):
                    tokens_removed += len(word_tokenize(line))
                if( line != "" and line[0] == '+'):
                    tokens_added += len(word_tokenize(line))
                k += 1

commit_msg = data['commit_metadata']['message'].split()
commit_msg_stemmed = [stemmer.stem(word) for word in commit_msg]
commit_msg_stemmed = ' '.join(commit_msg_stemmed)

sec_words_present = sec_words_present + [word for word in security_words if word in commit_msg_stemmed]
    

# print(tokens_added)
# print(tokens_removed)
# print(mem_words_present)
# print(sec_words_present)

In [19]:
def update_csv(json_file, csv_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
        file_changes = data['file_changes']
        tokens_added = 0
        tokens_removed = 0
        mem_words_present = []
        sec_words_present = []

        #tokens added or removed with memory related fxns
        for i in file_changes:
            for j in file_changes[i]:
                if j != 'used_context_lines':
                    k = 0
                    while k < len(file_changes[i][j]):
                        line = file_changes[i][j][k]
                        mem_words_present = mem_words_present + [word for word in memory_fxns if word in line]
                        if( line != "" and line[0] == '-'):
                            tokens_removed += len(word_tokenize(line))
                        if( line != "" and line[0] == '+'):
                            tokens_added += len(word_tokenize(line))
                        k += 1

        #security related keywords in commit msg
        commit_msg = data['commit_metadata']['message'].split()
        commit_msg_stemmed = [stemmer.stem(word) for word in commit_msg]
        commit_msg_stemmed = ' '.join(commit_msg_stemmed)

        sec_words_present = sec_words_present + [word for word in security_words if word in commit_msg_stemmed]
            
        #metadata
        metadata = data['commit_metadata']
        #basic counts
        author_length = len(metadata['author'])
        email_length = len(metadata['author_email'])
        msg_length = len(metadata['message'])
        #tokens
        # sec_words_present = [word for word in security_words if word in metadata['message'].lower()]
        # token_msg = word_tokenize(metadata['message'])
        # tokens = len(token_msg)
        df.loc[len(df.index)] = [author_length, email_length, msg_length, sec_words_present, mem_words_present, tokens_added, tokens_removed]
    #display(df)          
    df.to_csv(csv_file, index = False)
#test run
#update_csv('jsonfolder/CVE-1999-0199.json', 'character_counts.csv')

In [ ]:
json_folder = 'commit_metadata'
csv_file = 'character_counts.csv'
df = pd.DataFrame(columns = ['Length of author name', 'Length of author email', 'Length of commit message', 'Security related keywords in message', 'Memory related functions in code changes', 'Tokens added in commit', 'Tokens removed in commit'])
for file in os.listdir(json_folder):
    if file.endswith('.json'):
        json_file = os.path.join(json_folder, file)
        update_csv(json_file, csv_file)

In [ ]:
# ignore this block
# current_directory = 'commit_metadata'

# files = os.listdir(current_directory)

# for file in files:
#     if file.endswith('.json'):
#         file_path = os.path.join(current_directory, file)
#         os.remove(file_path)